<a href="https://colab.research.google.com/github/zbynekcz12/1/blob/main/decentralizovan%C3%A1%20s%C3%AD%C5%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import time
from datetime import datetime, timedelta

class EnergySource:
    def __init__(self, name, capacity):
        self.name = name
        self.capacity = capacity
        self.production = 0
        self.is_active = True
        self.maintenance_needed = False
        self.last_maintenance = datetime.now()
        self.maintenance_interval = timedelta(seconds=10)  # Předpokládaný interval údržby

    def generate_energy(self):
        if self.is_active:
            self.production = random.uniform(0, self.capacity)
        else:
            self.production = 0
        return self.production

    def disrupt(self):
        self.is_active = False
        print(f"{self.name} - Přerušení proudu!")

    def restore(self):
        self.is_active = True
        print(f"{self.name} - Spojení obnoveno.")

    def check_maintenance(self):
        if datetime.now() - self.last_maintenance >= self.maintenance_interval:
            self.maintenance_needed = True
            print(f"{self.name} - Potřeba údržby!")

    def perform_maintenance(self):
        self.maintenance_needed = False
        self.last_maintenance = datetime.now()
        print(f"{self.name} - Údržba provedena.")

class EnergyConsumer:
    def __init__(self, name, demand, battery_capacity, password):
        self.name = name
        self.demand = demand
        self.consumption = 0
        self.battery = BatteryStorage(battery_capacity)
        self.password = password
        self.is_connected = True
        self.monthly_consumption = 0
        self.energy_price = 0.2  # Cena za kWh

    def consume_energy(self, available_energy):
        if self.is_connected:
            self.consumption = min(available_energy, self.demand)
            self.battery.charge(self.consumption)
            self.monthly_consumption += self.consumption
        else:
            self.consumption = 0
        return self.consumption

    def print_battery_status(self):
        if self.is_connected:
            battery_percentage = (self.battery.stored_energy / self.battery.capacity) * 100
            print(f"{self.name} - Stav baterie: {self.battery.stored_energy:.2f} kWh ({battery_percentage:.2f}%)")

    def connect(self, password):
        if password == self.password:
            self.is_connected = True
            print(f"{self.name} - Připojeno.")
        else:
            print(f"{self.name} - Nesprávné heslo, připojení se nezdařilo.")

    def disconnect(self, password):
        if password == self.password:
            self.is_connected = False
            print(f"{self.name} - Odpojeno.")
        else:
            print(f"{self.name} - Nesprávné heslo, odpojení se nezdařilo.")

    def print_monthly_invoice(self):
        total_cost = self.monthly_consumption * self.energy_price
        print(f"\nMěsíční faktura pro {self.name}:")
        print(f"Spotřebovaná energie: {self.monthly_consumption:.2f} kWh")
        print(f"Cena za kWh: {self.energy_price:.2f} Kč")
        print(f"Celková cena: {total_cost:.2f} Kč")
        self.monthly_consumption = 0  # Reset měsíční spotřeby

class BatteryStorage:
    def __init__(self, capacity):
        self.capacity = capacity
        self.stored_energy = 0

    def charge(self, energy):
        self.stored_energy = min(self.capacity, self.stored_energy + energy)
        return self.stored_energy

    def discharge(self, demand):
        released_energy = min(self.stored_energy, demand)
        self.stored_energy -= released_energy
        return released_energy

class EnergyMarket:
    def __init__(self):
        self.transactions = []
        self.total_energy_traded = 0
        self.total_balance = 0
        self.monthly_transactions = []
        self.monthly_energy_traded = 0
        self.monthly_balance = 0
        self.last_month_end = datetime.now()

    def trade(self, source, consumer, energy):
        if energy > 0:
            transaction = (source.name, consumer.name, energy)
            self.transactions.append(transaction)
            self.monthly_transactions.append(transaction)
            self.total_energy_traded += energy
            self.total_balance += energy
            self.monthly_energy_traded += energy
            self.monthly_balance += energy
            return energy
        return 0

    def print_accounting_book(self):
        print("\nMalá účetní kniha pro simulovaný trh energie:")
        print(f"Celkové množství směněné energie: {self.total_energy_traded:.2f} kWh")
        print(f"Celkový zůstatek: {self.total_balance:.2f} kWh")

    def print_monthly_accounting_book(self):
        print("\nMěsíční závěrka účetní knihy pro simulovaný trh energie:")
        print(f"Celkové množství směněné energie za měsíc: {self.monthly_energy_traded:.2f} kWh")
        print(f"Celkový zůstatek za měsíc: {self.monthly_balance:.2f} kWh")
        self.monthly_transactions = []
        self.monthly_energy_traded = 0
        self.monthly_balance = 0
        self.last_month_end = datetime.now()

class EneXChange:
    def __init__(self):
        self.sources = []
        self.consumers = []
        self.battery = BatteryStorage(50)
        self.market = EnergyMarket()
        self.network = FiveGNetwork()
        self.start_time = datetime.now()

    def add_source(self, source):
        self.sources.append(source)

    def add_consumer(self, consumer):
        self.consumers.append(consumer)

    def balance_load(self, total_production, total_demand):
        if total_production > total_demand:
            surplus_energy = total_production - total_demand
            self.battery.charge(surplus_energy)
            print(f"Přebytek energie: {surplus_energy:.2f} kWh")
            print(f"Uložená energie v baterii: {self.battery.stored_energy:.2f} kWh")
        else:
            deficit_energy = total_demand - total_production
            released_energy = self.battery.discharge(deficit_energy)
            print(f"Nedostatek energie: {deficit_energy:.2f} kWh")
            print(f"Vybitá energie z baterie: {released_energy:.2f} kWh")

    def simulate(self):
        # Simulace náhodného přerušení proudu
        if random.random() < 0.1:  # 10% šance na přerušení
            for source in self.sources:
                source.disrupt()
        else:
            for source in self.sources:
                source.restore()

        # Kontrola a provedení prediktivní údržby
        for source in self.sources:
            source.check_maintenance()
            if source.maintenance_needed:
                source.perform_maintenance()

        total_production = sum(source.generate_energy() for source in self.sources)
        total_demand = sum(consumer.demand for consumer in self.consumers if consumer.is_connected)

        print(f"Celková produkce: {total_production:.2f} kWh")
        print(f"Celková poptávka: {total_demand:.2f} kWh")

        self.balance_load(total_production, total_demand)

        available_energy = total_production + self.battery.stored_energy
        for consumer in self.consumers:
            if consumer.is_connected:
                consumed_energy = consumer.consume_energy(available_energy)
                self.market.trade(self.sources[0], consumer, consumed_energy)
                available_energy -= consumed_energy

        for source in self.sources:
            print(f"{source.name} vyrobil {source.production:.2f} kWh")

        for consumer in self.consumers:
            consumer.print_battery_status()

        print("Transakce na trhu s energií:")
        for transaction in self.market.transactions:
            print(f"Zdroj: {transaction[0]}, Spotřebitel: {transaction[1]}, Energie: {transaction[2]:.2f} kWh")

        self.market.print_accounting_book()

        # Kontrola měsíční závěrky
        if datetime.now() >= self.start_time + timedelta(seconds=30):
            self.market.print_monthly_accounting_book()
            for consumer in self.consumers:
                consumer.print_monthly_invoice()
            self.start_time = datetime.now()

class FiveGNetwork:
    def __init__(self):
        self.connected_devices = []

    def connect(self, device):
        self.connected_devices.append(device)

    def broadcast(self, message):
        for device in self.connected_devices:
            print(f"Posílám zprávu '{message}' do {device.name}")

# Příklad použití
if __name__ == "__main__":
    eneXChange = EneXChange()

    # Přidání zdrojů energie
    eneXChange.add_source(EnergySource("Fotovoltaický panel 1", 10))
    eneXChange.add_source(EnergySource("Větrná turbína 1", 20))

    # Přidání spotřebitelů energie s vymyšlenými jmény a hesly
    eneXChange.add_consumer(EnergyConsumer("Jan Novák", 15, 20, "heslo1"))
    eneXChange.add_consumer(EnergyConsumer("Petra Svobodová", 10, 15, "heslo2"))
    eneXChange.add_consumer(EnergyConsumer("Jiří Novotný", 12, 18, "heslo3"))
    eneXChange.add_consumer(EnergyConsumer("Eva Dvořáková", 8, 10, "heslo4"))
    eneXChange.add_consumer(EnergyConsumer("Martin Vlček", 14, 22, "heslo5"))
    eneXChange.add_consumer(EnergyConsumer("Lucie Benešová", 9, 13, "heslo6"))
    eneXChange.add_consumer(EnergyConsumer("Michal Kovář", 11, 16, "heslo7"))
    eneXChange.add_consumer(EnergyConsumer("Kateřina Nová", 7, 9, "heslo8"))

    # Připojení spotřebitelů k 5G síti
    for consumer in eneXChange.consumers:
        eneXChange.network.connect(consumer)

    # Simulace výměny energie po dobu 30 vteřin
    start_time = time.time()
    while time.time() - start_time < 30:
        print("\nNová simulace:")
        eneXChange.simulate()
        eneXChange.network.broadcast("Aktualizace stavu")
        time.sleep(1)  # Počkej 1 sekundu před další simulací

        # Příklad odpojení a připojení domácnosti
        if random.random() < 0.2:  # 20% šance na změnu stavu
            consumer = random.choice(eneXChange.consumers)
            if consumer.is_connected:
                consumer.disconnect("nesprávné_heslo")  # Nesprávné heslo
                consumer.disconnect(consumer.password)  # Správné heslo
            else:
                consumer.connect("nesprávné_heslo")  # Nesprávné heslo
                consumer.connect(consumer.password)  # Správné heslo